In [8]:
import speech_recognition as sr
import wikipedia as wk
import webbrowser
import requests
import pandas as pd
import json
import pyttsx3
import time
import nltk
from nltk.corpus import wordnet

In [12]:
engine = pyttsx3.init()
engine.setProperty('rate',120)

r = sr.Recognizer()
with sr.Microphone() as source:
    print("Say Something:")
    audio = r.listen(source)
    text = r.recognize_google(audio)
    t=text.split(" ")
print(t)
if 'name' in t:
    engine.say("My name is Ananay Gupta")
    engine.runAndWait()
elif 'age' in t:
    engine.say("My age is 20 years")
    engine.runAndWait()
elif 'about' in t:
    i=t.index('about')
    print(wk.summary(t[i+1:]))
    engine.say(wk.summary(t[i+1:]))
    engine.runAndWait()
elif 'play' in t:
    i=t.index('play')
    name=t[i+1:]
    n=""
    for x in name:
        n=n+x+" "
    print(n)
    url='https://www.youtube.com/results?search_query='+n
    r=requests.get(url)

    from bs4 import BeautifulSoup
    soup=BeautifulSoup(r.text,'html.parser')
    soup.prettify

    l=[]
    for link in soup.find_all('a'):
        if(link['href'][:6]=='/watch'):
            l.append("https://www.youtube.com/"+link['href'])
    webbrowser.open(l[0])
elif 'temperature' in t:
    i=t.index('temperature')
    x=""+t[i+2]
    r=requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={x}&appid=94946c72f63e40971c24d47aa1f64fa2')
    d=json.loads(r.text)
    df=pd.DataFrame(d['main'],index=[1])
    df[['temp','feels_like','temp_min','temp_max']]=df[['temp','feels_like','temp_min','temp_max']]-273.15
    df['description']=d['weather'][0]['description']
    t=df['temp']
    t=round(t.values[0],1)
    engine.say(f"Temperature in {x} is {t} degree celsius")
    engine.runAndWait()
elif 'news' in t:
    r=requests.get('https://newsapi.org/v2/top-headlines?q=india&apiKey=c6eb1d3676144c0f8af0a52251950ac7')
    d=json.loads(r.text)
    df=pd.DataFrame(d['articles'])
    d1=df['description'].values
    for x in d1:
        print(x)
        engine.say(x)
        time.sleep(0.2)
        engine.runAndWait()
elif 'cases' in t:
    r=requests.get('https://api.covid19india.org/raw_data1.json')
    d=json.loads(r.text)
    df1=pd.DataFrame(d['raw_data'])
    r=requests.get('https://api.covid19india.org/raw_data2.json')
    d=json.loads(r.text)
    df2=pd.DataFrame(d['raw_data'])
    r=requests.get('https://api.covid19india.org/raw_data3.json')
    d=json.loads(r.text)
    df3=pd.DataFrame(d['raw_data'])
    del df1['backupnotes']
    del df2['backupnotes']
    df=df1.append(df3)
    df=df.append(df2)
    r=requests.get('https://api.covid19india.org/raw_data4.json')
    d=json.loads(r.text)
    df4=pd.DataFrame(d['raw_data'])
    df=df.append(df4)
    df['numcases'] = df['numcases'].apply(lambda x: '1' if x =='' else x)
    df['numcases']=pd.to_numeric(df['numcases'])
    df=df.drop(['source1','source2','source3','patientnumber','notes','estimatedonsetdate','entryid'],axis=1)
    g=df.dateannounced.str.split('/',expand=True).iloc[:,0:3]
    g.columns=['day','month','year']
    df=pd.concat([df,g],axis=1)
    d=df[df['currentstatus']=="Hospitalized"]
    i=t.index('cases')
    state=t[i+2]
    if 'India' in t:
        s=d.numcases.sum()
    else:
        d1=d[(d['detectedstate']==t[i+2])]
        s=d1.numcases.sum()
    engine.say(f"There are  total {s} cases in {state}")
    engine.runAndWait()
elif 'meaning' in t:
    i=t.index('meaning')
    word=t[i+2]
    SynArr=wordnet.synsets(word)
    syn=SynArr[0]
    print(syn.definition())
    engine.say(f"Meaning of {word} is {syn.definition()}")
    engine.runAndWait()
elif 'opposite' in t:
    i=t.index('opposite')
    word=t[i+2]
    SynArr=wordnet.synsets(word)
    print(SynArr[0].lemmas()[0].antonyms()[0].name())
    engine.say(f"Opposite of {word} is {SynArr[0].lemmas()[0].antonyms()[0].name()}")
    engine.runAndWait()

Say Something:
['what', 'is', 'the', 'opposite', 'of', 'beautiful']
ugly
